In [1]:
import thesis_helper
functions = thesis_helper.Thesis_Helper()
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [2]:


import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from flair.data import Sentence

from flair.embeddings import ELMoEmbeddings

# init embedding
embedding = ELMoEmbeddings('small')

In [3]:
annotations = '/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Combined/Taxonomy/Normal/Annotated/combined_annotations.csv'

In [4]:
df = pd.read_csv(annotations,sep=';')

#Filling any empty context columns with 'empty'
df['left_context'] = df['left_context'].fillna('empty')
df['right_context'] = df['right_context'].fillna('empty')

df['concatenated'] = df['left_context'] + ' | ' + df['candidate_skill'] + ' | ' + df['right_context']
print('Number of annotated rows ',df.shape[0])

Number of annotated rows  20836


In [5]:
def ELMo_embedder(text,label):

    label = label
    string = Sentence(text)
    embedding.embed(string)


    # Creating a list which stores the indexes of the | symbols
    bar_indexes_skill = []
    #Creating a list which stores the embeddings_tensors_skill
    embeddings_tensors_skill = []
    #Creating a list to store the labels
    labels = []

    #Checking the sentence object for the | symbols and storing their indexes
    for x in range(1,len(string)+1):
        if '|' in str(string.get_token(x)):
            bar_indexes_skill.append(x)

    token_list = []

    #Collecting the embeddings for every index between the indexes in bar_indexes_skill
    word_embedding_indexes = range(bar_indexes_skill[0],bar_indexes_skill[1]-1)
    for x in word_embedding_indexes:
        embeddings_tensors_skill.append(pd.Series(string[x].embedding))
        token_list.append(string[x].text)

    #Turning the elements from embeddings_tensors_skill into dataframe rows
    row_skill = pd.DataFrame()
    for x in range(0,len(embeddings_tensors_skill)):
        indi_row = pd.DataFrame(pd.Series(embeddings_tensors_skill[x]))
        indi_row = indi_row.transpose()
        row_skill = pd.concat([row_skill,indi_row])

    #Changing the column names in order to make pd.concat work later
    row_skill.columns = [x for x in range(0,len(row_skill.columns))]
    row_skill['label'] = label
    row_skill['token'] = token_list

    #This section converts all non labeled tokens into seperate bert embeddings

    # Creating a list which stores the indexes of the | symbols
    bar_indexes = []
    #Creating a list which stores the embedding_tensors
    embedding_tensors2 = []
    #Creating a list to store the labels
    labels = []

    #Checking the sentence object for the | symbols and storing their indexes
    for x in range(1,len(string)+1):
        if '|' in str(string.get_token(x)):
            bar_indexes.append(x)

    #Collecting the embeddings for every index between the indexes in bar_indexes
    word_embedding_indexes = range(bar_indexes[0]-1,bar_indexes[1])
    banaan = range(0,len(string))

    other_words = [x for x in banaan if x not in word_embedding_indexes]
    #other_words = [0,1]

    token_list = []

    for x in other_words:
        embedding_tensors2.append(pd.Series(string[x].embedding))
        token_list.append(string[x].text)


    #Turning the elements from embedding_tensors into dataframe rows
    row = pd.DataFrame()
    for x in range(0,len(embedding_tensors2)):
        indi_row = pd.DataFrame(pd.Series(embedding_tensors2[x]))
        indi_row = indi_row.transpose()
        row = pd.concat([row,indi_row])

    row['label'] = 0
    row['token'] = token_list

    final_row = pd.concat([row_skill,row]).reset_index(drop=True)
    return final_row



In [6]:
elmo_embeddings = pd.DataFrame()

for text,label in tqdm(zip(df['concatenated'],df['label'])):
    elmo_embeddings = pd.concat([elmo_embeddings,ELMo_embedder(text,label)])


20836it [1:43:22,  3.36it/s]


In [7]:
from sklearn.utils import shuffle

elmo_embeddings = shuffle(elmo_embeddings)

In [8]:
x_elmo = elmo_embeddings.drop(columns=['token','label'])

In [9]:
%%time

functions.model_performance(x_elmo, elmo_embeddings['label'])

  0%|          | 0/6 [00:00<?, ?it/s]

Starting model evaluation
We are at classifier  LogisticRegression(max_iter=10000000000000000000000, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 109.5min finished
 17%|█▋        | 1/6 [1:49:30<9:07:30, 6570.07s/it]

We are at classifier  GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 327.0min finished
 33%|███▎      | 2/6 [7:16:32<15:49:50, 14247.65s/it]

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 340.6min finished
 83%|████████▎ | 5/6 [19:56:35<4:38:30, 16710.99s/it]

We are at classifier  MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,),
              max_iter=10000000000000000000000, random_state=456,
              solver='lbfgs')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 236.1min finished
100%|██████████| 6/6 [23:52:43<00:00, 14327.23s/it]  


CPU times: user 5.41 s, sys: 22.1 s, total: 27.5 s
Wall time: 23h 52min 45s


,Classifier,Precision,Recall,F1
0,LR,0.792410,0.667257,0.717733
1,GBC,0.528836,0.364413,0.378067
2,SGD,0.787492,0.591499,0.648538
3,RF,0.931601,0.521168,0.593578
4,SVM,0.887527,0.590550,0.635641
5,MLP,0.694874,0.734249,0.712922


In [10]:
x_elmo = elmo_embeddings.reset_index(drop=True)

x_elmo['pos'] = x_elmo['token'].progress_apply(functions.pos_tagger)
x_elmo['pos'] = x_elmo['pos'].progress_apply(functions.sequence_counter)

pos_dicts = x_elmo[['pos']]
pos_dicts = pos_dicts['pos'].apply(pd.Series)
pos_dicts = pos_dicts.fillna(0).astype(int)

x_elmo['dep'] = x_elmo['token'].progress_apply(functions.dep_tagger)
x_elmo['dep'] = x_elmo['dep'].progress_apply(functions.sequence_counter)

dep_dicts = x_elmo[['dep']]
dep_dicts = dep_dicts['dep'].apply(pd.Series)
dep_dicts = dep_dicts.fillna(0).astype(int)

x_pos = pos_dicts.join(dep_dicts,lsuffix='_gram', rsuffix='_pos')

x = x_elmo.drop(columns=['token','label','pos','dep']).join(x_pos, lsuffix='_embedding', rsuffix='_pos')

100%|██████████| 205777/205777 [23:09<00:00, 148.06it/s]


In [11]:
%%time
functions.model_performance(x.fillna(0.0), elmo_embeddings['label'])

  0%|          | 0/6 [00:00<?, ?it/s]

Starting model evaluation
We are at classifier  LogisticRegression(max_iter=10000000000000000000000, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 133.8min finished
 17%|█▋        | 1/6 [2:13:46<11:08:50, 8026.14s/it]

We are at classifier  GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 839.0min finished
 33%|███▎      | 2/6 [16:12:46<36:34:27, 32916.82s/it]

...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 243.1min finished
 83%|████████▎ | 5/6 [22:14:16<3:52:29, 13949.70s/it]

We are at classifier  MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,),
              max_iter=10000000000000000000000, random_state=456,
              solver='lbfgs')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 104.5min finished
100%|██████████| 6/6 [23:58:48<00:00, 14388.04s/it]  


CPU times: user 5.66 s, sys: 20.1 s, total: 25.8 s
Wall time: 23h 58min 49s


,Classifier,Precision,Recall,F1
0,LR,0.792097,0.670080,0.719886
1,GBC,0.528836,0.364413,0.378067
2,SGD,0.799346,0.593939,0.646220
3,RF,0.926434,0.523284,0.596734
4,SVM,0.887442,0.590163,0.635371
5,MLP,0.702307,0.734510,0.717455
